In [1]:
%matplotlib inline
import os

In [2]:
os.environ['DJ_SUPPORT_ADAPTED_TYPES'] = 'TRUE'
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = 'TRUE'

In [3]:
import datajoint as dj

In [4]:
dj.config['database.host'] = 'workshop-db.datajoint.io'

In [6]:
dj.conn().connect()

Please enter DataJoint username: thinh
Please enter DataJoint password: ········
Connecting thinh@workshop-db.datajoint.io:3306


In [7]:
dj.config['enable_python_native_blobs'] = True
dj.config['display.limit'] = 5

In [8]:
import pynwb
from pynwb import NWBFile, NWBHDF5IO
from datetime import datetime
from dateutil.tz import tzlocal
import json
import numpy as np
import pathlib

In [9]:
lab = dj.create_virtual_module('lab', 'djneuro_sfn19_lab')
experiment = dj.create_virtual_module('experiment', 'djneuro_sfn19_experiment')
ephys = dj.create_virtual_module('ephys', 'djneuro_sfn19_ephys')
psth = dj.create_virtual_module('psth', 'djneuro_sfn19_psth')

In [10]:
import warnings
warnings.filterwarnings('ignore')

# Objective

This notebook presents a specific example usecase of incorporating ***NWB*** objects into DataJoint using the new ***dj.AttributeAdapter*** feature. 

We wish to create a table storing NWB object, with attribute type of ***filepath***. The idea is to generate NWB files, one for each session, that can be access from the file system, hence type ***filepath***, and can also be fetched and worked with as part of DatJoint pipeline. This can be accomplished with the new ***dj.AttributeAdapter*** feature.

As prerequisite, readers should be familiar with the concept and operation of ***dj.AttributeAdapter***, a review can be found [here](./Adapted-Types.ipynb).

## Step 0 - Create a ***store*** for the filepath

In [11]:
exported_nwb_dir = 'C:/Users/thinh/Documents/nwb_store'

In [12]:
dj.config['stores'] = {
    'nwbstore': {'protocol': 'file',
                 'stage': exported_nwb_dir,
                 'location': exported_nwb_dir}}

## Step 1 - Create a DataJoint AttributeAdapter for NWB object

Basically we will need to define an object inhereted from `dj.AttributeAdapter` and instantiated with a variable name ***nwb_obj***

In [13]:
class NWBAdapter(dj.AttributeAdapter):
    attribute_type = 'filepath@nwbstore'
    
    def put(self, nwb):
        save_file_name = ''.join([nwb.identifier, '.nwb'])
        with NWBHDF5IO(os.path.join(exported_nwb_dir, save_file_name), mode='w') as io:
            io.write(nwb)
            print(f'Write NWB 2.0 file: {save_file_name}')
        return os.path.join(exported_nwb_dir, save_file_name)
        
    def get(self, path):
        return NWBHDF5IO(path, mode='r').read()

#### Instantiate for use as a datajoint type

In [14]:
nwb_obj = NWBAdapter()

## Step 2 - Create a new schema ***export*** and NWB table

This ***NWB*** table specifies a primary key of `experiment.Session`, designed to store one NWB object (or NWBFile) per session

In [17]:
schema = dj.schema('djneuro_sfn19_export')

In [18]:
@schema
class NWB(dj.Manual):
    definition = """
    -> experiment.Session
    ---
    nwb: <nwb_obj> 
    """

Note that the table definition above set the ***nwb*** attribute to be of type ***< nwb_obj >***. 

Hence the reason for defining ***nwbfile*** as an instant of ***NWBAdapter*** - see Step 1

## Step 3 - Build an NWBFile and insert into ***NWB** table

Construct an NWBFile from the DataJoint pipeline is accomplished using an export function [here](https://github.com/ttngu207/Li-2015a/blob/master/pipeline/export/datajoint_to_nwb.py)

In [19]:
experiment.Session()

subject_id institution 6 digit animal ID,session session number,session_date,username,rig
210861,1,2013-07-01,Nuo Li,None
210861,2,2013-07-02,Nuo Li,None
210861,3,2013-07-03,Nuo Li,None
210862,1,2013-06-26,Nuo Li,None
210862,2,2013-06-27,Nuo Li,None


In [20]:
session_key = (experiment.Session & {'subject_id': '210861', 'session': 1}).fetch1('KEY')

### Construct an NWB object from the ***session_key***

This includes fetching the data from the pipeline pertained to this session, indicated by the ***session_key*** and construct an NWBFile using the ***pynwb*** package. Let's define a function ***export_to_nwb*** for that

In [21]:
zero_zero_time = datetime.strptime('00:00:00', '%H:%M:%S').time()  # no precise time available
hardware_filter = 'Bandpass filtered 300-6K Hz'
institution = 'Janelia Research Campus'

def export_to_nwb(session_key):
    this_session = (experiment.Session & session_key).fetch1()
    
    # ===============================================================================
    # =============================== META INFORMATION ==============================
    # ===============================================================================
    nwb = NWBFile(identifier='_'.join(
        ['ANM' + str(this_session['subject_id']),
         this_session['session_date'].strftime('%Y-%m-%d'),
         str(this_session['session'])]),
        session_description='',
        session_start_time=datetime.combine(this_session['session_date'], zero_zero_time),
        file_create_date=datetime.now(tzlocal()),
        experimenter=this_session['username'],
        institution=institution)
    # -- subject
    subj = (lab.Subject & session_key).fetch1()
    nwb.subject = pynwb.file.Subject(
        subject_id=str(this_session['subject_id']),
        genotype=' x '.join((lab.Subject.GeneModification
                             & subj).fetch('gene_modification')),
        sex=subj['sex'],
        species=subj['species'],
        date_of_birth=datetime.combine(subj['date_of_birth'], zero_zero_time) if subj['date_of_birth'] else None)

    # ===============================================================================
    # ======================== EXTRACELLULAR & CLUSTERING ===========================
    # ===============================================================================

    """
    In the event of multiple probe recording (i.e. multiple probe insertions), the clustering results 
    (and the associated units) are associated with the corresponding probe. 
    Each probe insertion is associated with one ElectrodeConfiguration (which may define multiple electrode groups)
    """

    dj_insert_location = ephys.ProbeInsertion.InsertionLocation * experiment.BrainLocation

    for probe_insertion in ephys.ProbeInsertion & session_key:
        electrode_config = (lab.ElectrodeConfig & probe_insertion).fetch1()

        electrode_groups = {}
        for electrode_group in lab.ElectrodeConfig.ElectrodeGroup & electrode_config:
            electrode_groups[electrode_group['electrode_group']] = nwb.create_electrode_group(
                name=electrode_config['electrode_config_name'] + '_g' + str(electrode_group['electrode_group']),
                description='N/A',
                device=nwb.create_device(name=electrode_config['probe']),
                location=json.dumps({k: str(v) for k, v in (dj_insert_location & session_key).fetch1().items()
                                     if k not in dj_insert_location.primary_key}))

        for chn in (lab.ElectrodeConfig.Electrode * lab.Probe.Electrode & electrode_config).fetch(as_dict=True):
            nwb.add_electrode(id=chn['electrode'],
                                  group=electrode_groups[chn['electrode_group']],
                                  filtering=hardware_filter,
                                  imp=-1.,
                                  x=chn['x_coord'] if chn['x_coord'] else np.nan,
                                  y=chn['y_coord'] if chn['y_coord'] else np.nan,
                                  z=chn['z_coord'] if chn['z_coord'] else np.nan,
                                  location=electrode_groups[chn['electrode_group']].location)

        # --- unit spike times ---
        nwb.add_unit_column(name='quality', description='unit quality from clustering')
        nwb.add_unit_column(name='posx', description='estimated x position of the unit relative to probe (0,0)')
        nwb.add_unit_column(name='posy', description='estimated y position of the unit relative to probe (0,0)')
        nwb.add_unit_column(name='amp', description='unit amplitude')
        nwb.add_unit_column(name='snr', description='unit signal-to-noise')
        nwb.add_unit_column(name='cell_type', description='cell type (e.g. fast spiking or pyramidal)')

        for unit in (ephys.Unit * ephys.UnitCellType & probe_insertion).fetch(as_dict=True):
            # make an electrode table region (which electrode(s) is this unit coming from)
            nwb.add_unit(id=unit['unit'],
                             electrodes=[unit['electrode']],
                             electrode_group=electrode_groups[unit['electrode_group']],
                             quality=unit['unit_quality'],
                             posx=unit['unit_posx'],
                             posy=unit['unit_posy'],
                             amp=unit['unit_amp'] if unit['unit_amp'] else np.nan,
                             snr=unit['unit_snr'] if unit['unit_amp'] else np.nan,
                             cell_type=unit['cell_type'],
                             spike_times=unit['spike_times'],
                             waveform_mean=np.mean(unit['waveform'], axis=0),
                             waveform_sd=np.std(unit['waveform'], axis=0))
    return nwb

### Let's generate the NWBFile from the selected `session_key`

In [22]:
nwb = export_to_nwb(session_key)

In [23]:
nwb


root <class 'pynwb.file.NWBFile'>
Fields:
  devices: {
    A4x8-5mm-100-200-177 <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    silicon32_g0 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'pynwb.core.DynamicTable'>
  experimenter: Nuo Li
  file_create_date: [datetime.datetime(2019, 11, 1, 11, 47, 29, 329250, tzinfo=tzlocal())]
  identifier: ANM210861_2013-07-01_1
  institution: Janelia Research Campus
  session_start_time: 2013-07-01 00:00:00-05:00
  subject: subject <class 'pynwb.file.Subject'>
  timestamps_reference_time: 2013-07-01 00:00:00-05:00
  units: units <class 'pynwb.misc.Units'>

### Check the units and spikes

In [24]:
# Check the units and spikes
nwb.units.to_dataframe().head(5)

,quality,posx,posy,amp,snr,cell_type,spike_times,electrodes,electrode_group,waveform_mean,waveform_sd
id,,,,,,,,,,,
1,good,300.0,-239.23,NaN,NaN,FS,"[193.50960493587493, 193.56806993984222, 193.8...","[(2, nan, nan, nan, -1.0, {""ml_location"": ""250...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[1.054545529579609e-05, 9.056840695049851e-06,...","[2.089032804260185e-05, 2.097320909876938e-05,..."
2,good,300.0,-739.23,NaN,NaN,Pyr,"[1330.3045341565037, 1330.4772328330898, 1330....","[(7, nan, nan, nan, -1.0, {""ml_location"": ""250...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.137148949953386e-05, 2.2598475776179362e-05...","[1.151541178789538e-05, 1.1761191643470204e-05..."
3,good,100.0,-239.23,NaN,NaN,Pyr,"[1039.4155255987548, 1112.7271499458618, 1113....","[(10, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[6.566261472471525e-05, 6.708318969029598e-05,...","[2.532670367181818e-05, 2.5250325224515877e-05..."
5,good,100.0,-239.23,NaN,NaN,Pyr,"[122.88458597019958, 124.14954411343383, 124.1...","[(10, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[-1.829192958687734e-06, -1.3189889352710687e-...","[2.5708487505186185e-05, 2.5908135210449323e-0..."
6,good,100.0,-339.23,NaN,NaN,Pyr,"[441.3197799898224, 580.3622762567215, 611.775...","[(11, nan, nan, nan, -1.0, {""ml_location"": ""25...",\nsilicon32_g0 <class 'pynwb.ecephys.Electrode...,"[2.1640630960186472e-05, 2.3090471494022005e-0...","[1.730605003883158e-05, 1.6574918056606448e-05..."


### Insert to the ***NWB*** table

In [25]:
NWB.insert1({**session_key, 'nwb': nwb})

Write NWB 2.0 file: ANM210861_2013-07-01_1.nwb


In [26]:
NWB()

subject_id institution 6 digit animal ID,session session number,nwb
210861,1,=BLOB=


### Fetch to confirm

In [27]:
fetched_nwb = (NWB & session_key).fetch1('nwb')

In [28]:
fetched_nwb

"\nroot <class 'pynwb.file.NWBFile'>\nFields:\n  devices: {\n    A4x8-5mm-100-200-177 <class 'pynwb.device.Device'>\n  }\n  electrode_groups: {\n    silicon32_g0 <class 'pynwb.ecephys.ElectrodeGroup'>\n  }\n  electrodes: electrodes <class 'pynwb.core.DynamicTable'>\n  experimenter: ['Nuo Li']\n  file_create_date: [datetime.datetime(2019, 11, 1, 11, 47, 29, 329250, tzinfo=tzoffset(None, -18000))]\n  identifier: ANM210861_2013-07-01_1\n  institution: Janelia Research Campus\n  session_start_time: 2013-07-01 00:00:00-05:00\n  subject: subject <class 'pynwb.file.Subject'>\n  timestamps_reference_time: 2013-07-01 00:00:00-05:00\n  units: units <class 'pynwb.misc.Units'>\n"

### Confirmed the fetched NWB file ***units*** table

In [29]:
fetched_nwb.units.to_dataframe()

AttributeError: 'str' object has no attribute 'units'

## One step further, let's turn this routine into a dj.Computed table

With this definition and the ***make()*** logic, one NWBFile will be created, and inserted into the ***ComputedNWB*** table per session. With the attribute type of ***filepath***, the generated nwb files (.nwb) can be found at ***nwbstore***, with location defined as ***'/home/ttngu207/data'*** above

In [ ]:
@export
class ComputedNWB(dj.Computed):
    definition = """
    -> experiment.Session
    ---
    nwb: <nwb_obj> 
    """
    
    def make(self, key):
        nwbfile = export_to_nwb(key)
        self.insert1({**key, 'nwb': nwbfile})

In [ ]:
ComputedNWB.populate()